<a href="https://colab.research.google.com/github/Saisuhaschandra/AIML-LAB/blob/main/Assignment3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
{
 "cells": [
  {
   "cell_type": "markdown",
   "id": "126d57d5",
   "metadata": {},
   "source": [
    "## Implement Alpha-Beta Search Tree using Game Strategy\n",
    "### Prof.Kiran Eranki"
   ]
  },
  {
   "cell_type": "markdown",
   "id": "f0a894b4",
   "metadata": {},
   "source": [
    "## Part-01: List of Tasks to Perform\n",
    "\n",
    "1. Install the Python Libraries for Game Strategy\n",
    "2. Implement a Game Class Constructor with - actions, is_terminal, result, utility\n",
    "3. Implement a Player Game using game class function"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 24,
   "id": "2858d274",
   "metadata": {},
   "outputs": [],
   "source": [
    "from collections import namedtuple, Counter, defaultdict\n",
    "import random\n",
    "import math\n",
    "import functools \n",
    "cache = functools.lru_cache(10**6)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 25,
   "id": "1af90cb7",
   "metadata": {},
   "outputs": [],
   "source": [
    "class Game:\n",
    "    \"\"\"A game is similar to a problem, but it has a terminal test instead of \n",
    "    a goal test, and a utility for each terminal state. To create a game, \n",
    "    subclass this class and implement `actions`, `result`, `is_terminal`, \n",
    "    and `utility`. You will also need to set the .initial attribute to the \n",
    "    initial state; this can be done in the constructor.\"\"\"\n",
    "\n",
    "    def actions(self, state):\n",
    "        \"\"\"Return a collection of the allowable moves from this state.\"\"\"\n",
    "        raise NotImplementedError\n",
    "\n",
    "    def result(self, state, move):\n",
    "        \"\"\"Return the state that results from making a move from a state.\"\"\"\n",
    "        raise NotImplementedError\n",
    "\n",
    "    def is_terminal(self, state):\n",
    "        \"\"\"Return True if this is a final state for the game.\"\"\"\n",
    "        return not self.actions(state)\n",
    "    \n",
    "    def utility(self, state, player):\n",
    "        \"\"\"Return the value of this final state to player.\"\"\"\n",
    "        raise NotImplementedError"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 26,
   "id": "fc3e467e",
   "metadata": {},
   "outputs": [],
   "source": [
    "def play_game(game, strategies: dict, verbose=False):\n",
    "    \"\"\"Play a turn-taking game. `strategies` is a {player_name: function} dict,\n",
    "    where function(state, game) is used to get the player's move.\"\"\"\n",
    "    state = game.initial\n",
    "    while not game.is_terminal(state):\n",
    "        player = state.to_move\n",
    "        move = strategies[player](game, state)\n",
    "        state = game.result(state, move)\n",
    "        if verbose: \n",
    "            print('Player', player, 'move:', move)\n",
    "            print(state)\n",
    "    return state"
   ]
  },
  {
   "cell_type": "markdown",
   "id": "ee4a50ba",
   "metadata": {},
   "source": [
    "## Part-02: Implementation of Game Strategy Algorithm\n",
    "1. MiniMax Tree \n",
    "2. Alpha-Beta Search Algorithm"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 27,
   "id": "41bf6fe8",
   "metadata": {},
   "outputs": [],
   "source": [
    "def minimax_search(game, state):\n",
    "    \"\"\"Search game tree to determine best move; return (value, move) pair.\"\"\"\n",
    "\n",
    "    player = state.to_move\n",
    "\n",
    "    def max_value(state):\n",
    "        if game.is_terminal(state):\n",
    "            return game.utility(state, player), None\n",
    "        v, move = -infinity, None\n",
    "        for a in game.actions(state):\n",
    "            v2, _ = min_value(game.result(state, a))\n",
    "            if v2 > v:\n",
    "                v, move = v2, a\n",
    "        return v, move\n",
    "\n",
    "    def min_value(state):\n",
    "        if game.is_terminal(state):\n",
    "            return game.utility(state, player), None\n",
    "        v, move = +infinity, None\n",
    "        for a in game.actions(state):\n",
    "            v2, _ = max_value(game.result(state, a))\n",
    "            if v2 < v:\n",
    "                v, move = v2, a\n",
    "        return v, move\n",
    "\n",
    "    return max_value(state)\n",
    "\n",
    "infinity = math.inf"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 18,
   "id": "1b54aa9b",
   "metadata": {},
   "outputs": [],
   "source": [
    "def alphabeta_search(game, state):\n",
    "    \"\"\"Search game to determine best action; use alpha-beta pruning.\n",
    "    \"\"Search all the way to the leaves.\"\"\"\n",
    "\n",
    "    player = state.to_move\n",
    "\n",
    "    def max_value(state, alpha, beta):\n",
    "        if game.is_terminal(state):\n",
    "            return game.utility(state, player), None\n",
    "        v, move = -infinity, None\n",
    "        for a in game.actions(state): \n",
    "            v2, _ = min_value(game.result(state, a), alpha, beta)\n",
    "            if v2 > v:\n",
    "                v, move = v2, a\n",
    "                alpha = max(alpha, v)\n",
    "            if v >= beta:\n",
    "                return v, move\n",
    "        return v, move\n",
    "\n",
    "    def min_value(state, alpha, beta):\n",
    "        if game.is_terminal(state):\n",
    "            return game.utility(state, player), None\n",
    "        v, move = +infinity, None\n",
    "        for a in game.actions(state):\n",
    "            v2, _ = max_value(game.result(state, a), alpha, beta)\n",
    "            if v2 < v:\n",
    "                v, move = v2, a\n",
    "                beta = min(beta, v)\n",
    "            if v <= alpha:\n",
    "                return v, move\n",
    "        return v, move\n",
    "\n",
    "    return max_value(state, -infinity, +infinity)"
   ]
  },
  {
   "cell_type": "markdown",
   "id": "89ee8c76",
   "metadata": {},
   "source": [
    "## Part-03: Implement the Game Strategy using TicTacToe "
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 19,
   "id": "2b51f801",
   "metadata": {},
   "outputs": [],
   "source": [
    "class TicTacToe(Game):\n",
    "    \"\"\"Play TicTacToe on an `height` by `width` board, needing `k` in a row to win.\n",
    "    'X' plays first against 'O'.\"\"\"\n",
    "\n",
    "    def __init__(self, height=3, width=3, k=3):\n",
    "        self.k = k # k in a row\n",
    "        self.squares = {(x, y) for x in range(width) for y in range(height)}\n",
    "        self.initial = Board(height=height, width=width, to_move='X', utility=0)\n",
    "\n",
    "    def actions(self, board):\n",
    "        \"\"\"Legal moves are any square not yet taken.\"\"\"\n",
    "        return self.squares - set(board)\n",
    "\n",
    "    def result(self, board, square):\n",
    "        \"\"\"Place a marker for current player on square.\"\"\"\n",
    "        player = board.to_move\n",
    "        board = board.new({square: player}, to_move=('O' if player == 'X' else 'X'))\n",
    "        win = k_in_row(board, player, square, self.k)\n",
    "        board.utility = (0 if not win else +1 if player == 'X' else -1)\n",
    "        return board\n",
    "\n",
    "    def utility(self, board, player):\n",
    "        \"\"\"Return the value to player; 1 for win, -1 for loss, 0 otherwise.\"\"\"\n",
    "        return board.utility if player == 'X' else -board.utility\n",
    "\n",
    "    def is_terminal(self, board):\n",
    "        \"\"\"A board is a terminal state if it is won or there are no empty squares.\"\"\"\n",
    "        return board.utility != 0 or len(self.squares) == len(board)\n",
    "\n",
    "    def display(self, board): print(board)     \n",
    "\n",
    "\n",
    "def k_in_row(board, player, square, k):\n",
    "    \"\"\"True if player has k pieces in a line through square.\"\"\"\n",
    "    def in_row(x, y, dx, dy): return 0 if board[x, y] != player else 1 + in_row(x + dx, y + dy, dx, dy)\n",
    "    return any(in_row(*square, dx, dy) + in_row(*square, -dx, -dy)-1>=k\n",
    "               for (dx, dy) in ((0, 1), (1, 0), (1, 1), (1, -1)))\n"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 20,
   "id": "2425de7b",
   "metadata": {},
   "outputs": [],
   "source": [
    "class Board(defaultdict):\n",
    "    \"\"\"A board has the player to move, a cached utility value, \n",
    "    and a dict of {(x, y): player} entries, where player is 'X' or 'O'.\"\"\"\n",
    "    empty = '.'\n",
    "    off = '#'\n",
    "    \n",
    "    def __init__(self, width=8, height=8, to_move=None, **kwds):\n",
    "        self.__dict__.update(width=width, height=height, to_move=to_move, **kwds)\n",
    "        \n",
    "    def new(self, changes: dict, **kwds) -> 'Board':\n",
    "        \"Given a dict of {(x, y): contents} changes, return a new Board with the changes.\"\n",
    "        board = Board(width=self.width, height=self.height, **kwds)\n",
    "        board.update(self)\n",
    "        board.update(changes)\n",
    "        return board\n",
    "\n",
    "    def __missing__(self, loc):\n",
    "        x, y = loc\n",
    "        if 0 <= x < self.width and 0 <= y < self.height:\n",
    "            return self.empty\n",
    "        else:\n",
    "            return self.off\n",
    "            \n",
    "    def __hash__(self): \n",
    "        return hash(tuple(sorted(self.items()))) + hash(self.to_move)\n",
    "    \n",
    "    def __repr__(self):\n",
    "        def row(y): return ' '.join(self[x, y] for x in range(self.width))\n",
    "        return '\\n'.join(map(row, range(self.height))) +  '\\n'"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 21,
   "id": "49e7b830",
   "metadata": {},
   "outputs": [],
   "source": [
    "def random_player(game, state): return random.choice(list(game.actions(state)))\n",
    "\n",
    "def player(search_algorithm):\n",
    "    \"\"\"A game player who uses the specified search algorithm\"\"\"\n",
    "    return lambda game, state: search_algorithm(game, state)[1]"
   ]
  },
  {
   "cell_type": "markdown",
   "id": "f1f77e18",
   "metadata": {},
   "source": [
    "## Evaluate the Game Strategy using TicTokToe"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 22,
   "id": "f3558295",
   "metadata": {
    "scrolled": True
   },
   "outputs": [
    {
     "name": "stdout",
     "output_type": "stream",
     "text": [
      "Player X move: (1, 0)\n",
      ". X .\n",
      ". . .\n",
      ". . .\n",
      "\n",
      "Player O move: (1, 2)\n",
      ". X .\n",
      ". . .\n",
      ". O .\n",
      "\n",
      "Player X move: (1, 1)\n",
      ". X .\n",
      ". X .\n",
      ". O .\n",
      "\n",
      "Player O move: (0, 0)\n",
      "O X .\n",
      ". X .\n",
      ". O .\n",
      "\n",
      "Player X move: (0, 1)\n",
      "O X .\n",
      "X X .\n",
      ". O .\n",
      "\n",
      "Player O move: (2, 1)\n",
      "O X .\n",
      "X X O\n",
      ". O .\n",
      "\n",
      "Player X move: (0, 2)\n",
      "O X .\n",
      "X X O\n",
      "X O .\n",
      "\n",
      "Player O move: (2, 0)\n",
      "O X O\n",
      "X X O\n",
      "X O .\n",
      "\n",
      "Player X move: (2, 2)\n",
      "O X O\n",
      "X X O\n",
      "X O X\n",
      "\n"
     ]
    },
    {
     "data": {
      "text/plain": [
       "0"
      ]
     },
     "execution_count": 22,
     "metadata": {},
     "output_type": "execute_result"
    }
   ],
   "source": [
    "play_game(TicTacToe(), dict(X=random_player, O=player(alphabeta_search)), verbose=True).utility"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 23,
   "id": "80d941f6",
   "metadata": {},
   "outputs": [
    {
     "name": "stdout",
     "output_type": "stream",
     "text": [
      "Player X move: (0, 1)\n",
      ". . .\n",
      "X . .\n",
      ". . .\n",
      "\n",
      "Player O move: (2, 1)\n",
      ". . .\n",
      "X . O\n",
      ". . .\n",
      "\n",
      "Player X move: (1, 2)\n",
      ". . .\n",
      "X . O\n",
      ". X .\n",
      "\n",
      "Player O move: (0, 0)\n",
      "O . .\n",
      "X . O\n",
      ". X .\n",
      "\n",
      "Player X move: (1, 1)\n",
      "O . .\n",
      "X X O\n",
      ". X .\n",
      "\n",
      "Player O move: (1, 0)\n",
      "O O .\n",
      "X X O\n",
      ". X .\n",
      "\n",
      "Player X move: (2, 0)\n",
      "O O X\n",
      "X X O\n",
      ". X .\n",
      "\n",
      "Player O move: (0, 2)\n",
      "O O X\n",
      "X X O\n",
      "O X .\n",
      "\n",
      "Player X move: (2, 2)\n",
      "O O X\n",
      "X X O\n",
      "O X X\n",
      "\n"
     ]
    },
    {
     "data": {
      "text/plain": [
       "0"
      ]
     },
     "execution_count": 23,
     "metadata": {},
     "output_type": "execute_result"
    }
   ],
   "source": [
    "play_game(TicTacToe(), dict(X=player(alphabeta_search), O=player(minimax_search)), verbose=True).utility"
   ]
  }
 ],
 "metadata": {
  "kernelspec": {
   "display_name": "Python 3 (ipykernel)",
   "language": "python",
   "name": "python3"
  },
  "language_info": {
   "codemirror_mode": {
    "name": "ipython",
    "version": 3
   },
   "file_extension": ".py",
   "mimetype": "text/x-python",
   "name": "python",
   "nbconvert_exporter": "python",
   "pygments_lexer": "ipython3",
   "version": "3.9.15"
  }
 },
 "nbformat": 4,
 "nbformat_minor": 5
}
